<a href="https://colab.research.google.com/github/Ramanarayanan/DeepLearningAndGenerativeAi/blob/main/Copy_of_Fine_Tune_gpt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers Datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import json

from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments,DataCollatorForLanguageModeling
from datasets import Dataset
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
from datasets import load_dataset

In [ ]:
def load_dataset(file_path):
    data = []
    with open(file_path, 'r') as file:
        json_data = json.load(file)

        # for line in file:
        #     line = line.strip()  # Remove leading/trailing whitespace
        #     if line:  # Skip empty lines
        #         try:
        #             data.append(json.loads(line))
        #         except json.JSONDecodeError as e:
        #             print(f"Error decoding JSON in line: {line}")
        #             print(f"Error message: {e}")
        #             # You can choose to skip the line or raise the error
        #             # raise e  # Uncomment to raise the error and stop execution

    formatted_data = []
    for item in json_data:
        question = item['question']
        context = item['context']
        input_text = f"Question: {question}\nContext: {context}\nAnswer:"
        formatted_data.append({
            'input_text': input_text,
            'target_text': context
        })

    return Dataset.from_dict({'text': [item['input_text'] for item in formatted_data],
                              'labels': [item['target_text'] for item in formatted_data]})

In [ ]:

dataset = load_dataset('/content/SamplwCustomDataset_gpt2.json')  # Our data file name is 'data.jsonl'

In [ ]:
def tokenize_function(examples):
    tokenizer.pad_token = tokenizer.eos_token
    tokenized_inputs = tokenizer(examples['text'], truncation=True, padding='max_length', max_length=512)

    # Also tokenize labels and add them to the output
    # Assuming 'labels' contains the target text
    tokenized_labels = tokenizer(examples['labels'], truncation=True, padding='max_length', max_length=512)
    tokenized_inputs['labels'] = tokenized_labels['input_ids']
    return tokenized_inputs

In [ ]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/15 [00:00<?, ? examples/s]

In [ ]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)


In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="no",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    save_steps=10_000,
    save_total_limit=2,
    logging_steps=200,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer.train()


Step,Training Loss


TrainOutput(global_step=12, training_loss=3.573367436726888, metrics={'train_runtime': 20.4866, 'train_samples_per_second': 2.197, 'train_steps_per_second': 0.586, 'total_flos': 11758141440000.0, 'train_loss': 3.573367436726888, 'epoch': 3.0})

In [ ]:
model.save_pretrained("./trained-model")
tokenizer.save_pretrained("./trained-model")

('./trained-model/tokenizer_config.json',
 './trained-model/special_tokens_map.json',
 './trained-model/vocab.json',
 './trained-model/merges.txt',
 './trained-model/added_tokens.json')

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('./trained-model')
model = GPT2LMHeadModel.from_pretrained('./trained-model')

tokenizer.pad_token = tokenizer.eos_token

In [ ]:
contexts = []
with open("/content/SamplwCustomDataset_gpt2.json", "r") as file:
    json_data1 = json.load(file)
    for item in json_data1:

        contexts.append(item['context'])


In [ ]:
vectorizer = TfidfVectorizer()
context_vectors = vectorizer.fit_transform(contexts)

In [ ]:
def prepare_input(question, context):
    return f"Context: {context}\nQuestion: {question}\nAnswer:"

In [ ]:
def find_most_relevant_context(user_input, top_n=3):
    input_vector = vectorizer.transform([user_input])
    similarities = cosine_similarity(input_vector, context_vectors).flatten()
    most_similar_indices = np.argsort(similarities)[-top_n:][::-1]
    relevant_contexts = [contexts[idx] for idx in most_similar_indices]
    return list(dict.fromkeys(relevant_contexts))

In [ ]:
def generate_answer_gpt2(question, context):
    input_text = prepare_input(question, context)

    inputs = tokenizer.encode_plus(
        question,
        return_tensors='pt',
        padding=True,
        truncation=True,
        max_length=256
    )
    # inputs = tokenizer.encode_plus(
    #     input_text,
    #     return_tensors='pt',
    #     padding=True,
    #     truncation=True,
    #     max_length=256
    # )

    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']
    print(input_ids)
    # Top-K and Top-P Sampling Text Generation.
    # We can use different text generation technique like Top-P, Top-K,
    # greedy search, beam search or random search for different kind of response

    outputs = model.generate(
        input_ids=input_ids,
        max_new_tokens=150,
        attention_mask=attention_mask,
        do_sample=True,
        top_k=30,
        top_p=0.9,
        num_return_sequences=1
    )

    answer = tokenizer.decode(outputs[0], skip_special_tokens=False)
    if "Answer:" in answer:
        answer1 = answer.split("Answer:")[-1].strip()
        answer2 = answer.split("Answer:")[1].strip()  # This line should now be safe

        return answer1 if len(answer1) > len(answer2) else answer2
    else:
        # Handle the case where "Answer:" is not found
        print("Warning: 'Answer:' not found in generated response. Returning the full response.")
        return answer  # Or return a default value, or raise an exception

In [ ]:
user_input = "Therapy Support Suite"

relevant_context = find_most_relevant_context(user_input)
print(relevant_context)
all_context = " ".join(relevant_context)

response = generate_answer_gpt2(user_input, all_context)

print(response)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['At a glance The Therapy Support Suite (TSS) is the central clinical management software within TDMS. As such it supports the user in carrying out, creating and managing patient prescriptions, medication plans, documentation of treatment data, and patient-related laboratory data. Furthermore, it provides enhanced reporting capabilities and allows the professional management of an association of dialysis centers.', 'PD management software designed for your teamIndividual prescription support,Treatment results & analysis,Patient follow up by in-depth overview of PD therapy', 'If your item arrives damaged, please contact customer support within 7 days for a return or replacement.']
tensor([[35048, 12826,  7929, 26264]])
Therapy Support Suite 2 and a new app (called Support Services).

What you need:

The latest Android Studio version for Windows

An Android Development Kit (DLL) from Microsoft.

A Windows 10 app from Microsoft.

The App Launcher for Windows

A Windows 10-based Google App